# LDA

#### LDA Details
- Topic Modelling
    - topic = a repeating pattern of co-occurring terms in a corpus
- Parameters of LDA
    - Alpha = document <-> topic density
    - Beta = topic <-> word density
    - Higher alpha = more topics
    - High beta = larger number of words
- LDA Steps
    1. Prepare documents (put in list)
    2. Clean & Preprocess
        - remove punctuation and stopwords
        - convert word to root word (running and ran would be both converted to run)
    3. Prepare Document-Term Matrix
        - create dictionary from corpus (literally use gensim dictionary, it is the easiest way)
    4. Run LDA Model
        - create object for LDA and train on document-term matrix
        - fun fact-- gensim has an open-source LDA model that we can use!
    5. Get results!
        - To improve results
            - Frequency Filter (filter out words used less than 3 times and the top 10-20% of words)
            - Parts of Speech tag filter (in our case I think this would be a bad thing)
            - Base-wise LDA (we can look into this later)

In [1]:
# Imports that will be necessary

import re
import numpy as np
import pandas as pd
from pprint import pprint
import json

# ML imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

# for lemmatization -- but can't we use NLTK lemmatizer?
from nltk.stem import WordNetLemmatizer

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
# Preprocess Data

subreddits = ['depression', 'Anxiety', 'foreveralone', 'socialanxiety', 'SuicideWatch', 
                    'berkeley', 'PowerLedger', 'TalesFromYourServer', 'tifu']
total = []
temp = []
for subreddit in subreddits:
    with open('datasets/'+subreddit+'_text_samples_extended.json') as f:
        print('opening' + ' datasets/'+subreddit+'_text_samples_extended.json')
        data = json.load(f)
    
    for x in range(0, len(data['data'])):
        if x % 100 == 0:
            print(x)
        if 'selftext' in data['data'][x]:
            # filter out newline at a later time
            total.append(gensim.utils.simple_preprocess(data.get('data')[x].get('selftext')))            

opening datasets/depression_text_samples_extended.json
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
opening datasets/Anxiety_text_samples_extended.json
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
23

In [4]:
print(total[0][0])

cant


In [5]:
from autocorrect import spell

for x in range(len(total)):
    if x%100 == 0:
        print(x)
    for y in range(len(total[x])):
        total[x][y] = spell(total[x][y])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

133000
133100
133200
133300
133400
133500
133600
133700
133800
133900
134000
134100
134200
134300
134400
134500
134600
134700
134800
134900


In [6]:
# bigram and trigram model
bigram = gensim.models.Phrases(total, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[total], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[total[0]]])

['cant', 'keep', 'the', 'thought', 'of', 'potential', 'rape', 'out', 'of', 'my', 'head', 'every', 'single', 'day', 'know', 'men', 'get', 'raped', 'too', 'but', 'women', 'have', 'higher', 'chance', 'of', 'it', 'happening', 'to', 'them', 'men', 'in', 'general', 'dont', 'go', 'around', 'living', 'in', 'fear', 'of', 'being', 'raped', 'or', 'being', 'physically', 'vulnerable', 'when', 'they', 'step', 'outside', 'their', 'house', 'know', 'rape_isnt_womans', 'fault', 'and', 'rapist', 'are', 'just', 'gonna', 'be', 'rapist', 'but', 'just', 'cant', 'help', 'the', 'fact', 'that', 'rather', 'just', 'not', 'be', 'born', 'girl', 'to', 'even', 'have', 'to', 'deal', 'with', 'this', 'thought', 'of', 'rape', 'and', 'worrying', 'about', 'rape', 'everyday', 'and', 'getting', 'pregnant', 'men', 'dont', 'ever', 'worry', 'about', 'getting', 'pregnant', 'at', 'all', 'no', 'matter', 'what', 'hate', 'that', 'have', 'to', 'take', 'birth', 'control', 'if', 'rape', 'were', 'to', 'happen', 'or', 'to', 'just', 'be',

In [7]:
# get rid of stopwords, make bigrams and trigrams from our data

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
data_words_nostops = remove_stopwords(total)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['can', 'not', 'keep', 'think', 'potential', 'rape', 'head', 'single', 'day', 'know', 'man', 'get', 'rap', 'woman', 'high', 'chance', 'happen', 'man', 'general', 'do', 'not', 'go', 'live', 'fear', 'rap', 'physically', 'vulnerable', 'step', 'house', 'know', 'rape', 'isnt_woman', 'fault', 'rapist', 'go', 'rapist', 'can', 'not', 'help', 'fact', 'rather', 'bear', 'girl', 'even', 'deal', 'think', 'rape', 'worry', 'rape', 'everyday', 'get', 'pregnant', 'man', 'do', 'not', 'ever', 'worry', 'get', 'pregnant', 'matter', 'hate', 'take', 'birth', 'control', 'rape', 'happen', 'relationship', 'man', 'hate', 'physically', 'vulnerable', 'matter', 'many', 'weight', 'lift', 'martial_art', 'take', 'vulnerable', 'potential', 'get', 'rap', 'day', 'bear', 'day', 'die', 'especially', 'become', 'old', 'do', 'not', 'get', 'woman', 'deal', 'maybe', 'support', 'friend', 'especially', 'male', 'friend', 'protect', 'small', 'family', 'brother', 'mom', 'dad', 'probably', 'do', 'not', 'care', 'friend', 'difficult',

### This is where percent words removed can be edited

In [22]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Remove all words that are used less than 3 times.
# Remove top 10% of words? Less? More?
top_5_percent_as_number = int(len(id2word)*0.05)

id2word.filter_extremes(no_below = 3)
id2word.filter_n_most_frequent(top_5_percent_as_number)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 4), (7, 6), (8, 2), (9, 1), (10, 3)]]


In [23]:
id2word[0]

'birth'

In [24]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('birth', 1),
  ('isnt_woman', 1),
  ('lift', 1),
  ('martial_art', 1),
  ('pregnant', 2),
  ('protect', 1),
  ('rap', 4),
  ('rape', 6),
  ('rapist', 2),
  ('safety', 1),
  ('vulnerable', 3)]]

In [26]:
# Build LDA model
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=20, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)

In [27]:
# Print the Keyword in the 10 topics
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [28]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

In [29]:
# Improvement with mallet

mallet_path = 'LDA/mallet-2.0.8/bin/mallet'

In [30]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=15, id2word=id2word)

In [31]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, 
                                           dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(1,
  [('wrist', 0.009334707376695586),
   ('haunt', 0.0077649427215644926),
   ('pray', 0.007645113358577386),
   ('screen', 0.007441403441499305),
   ('faith', 0.007405454632603173),
   ('engage', 0.007129847097732829),
   ('coward', 0.006243109811628241),
   ('universe', 0.006063365767147582),
   ('military', 0.006063365767147582),
   ('facebook', 0.005907587595264344)]),
 (6,
  [('http_www', 0.01722707764807713),
   ('romantic', 0.012334250988783766),
   ('bf', 0.010711323227101692),
   ('instagram', 0.008294964115263936),
   ('tho', 0.008138682182657514),
   ('flat', 0.0072851423968839785),
   ('pussy', 0.006792253224817571),
   ('wtf', 0.006792253224817571),
   ('frustrate', 0.006431602611110443),
   ('ppl', 0.006323407426998305)]),
 (7,
  [('wellbutrin', 0.012294933326885646),
   ('winter', 0.008535125790345394),
   ('irl', 0.00831751635093149),
   ('doc', 0.008051549258314495),
   ('overdose', 0.008027370431712949),
   ('zoloft', 0.0072415585671627355),
   ('sexual', 0.0067942

In [32]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, 
                                                 num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, 
                             texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

Coherence Values and Optimal Topics:
- After removing the top 10% of words and words appearing less than 3 times
    - Coherence = 69% for 20 topics
    - Highest coherence is 73% for 2 topics (not a big fan of that, that's not useful)
- After removing the topic 5% of words and words appearing less than 3 times
    - 

In [43]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
vis

/home/athina/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
2     -0.212234 -0.299925       1        1  9.999410
1     -0.291780  0.230150       2        1  9.703836
8     -0.041874 -0.007609       3        1  8.597857
12    -0.019052 -0.002728       4        1  6.994777
14     0.006901  0.001659       5        1  6.723743
0      0.020020  0.003499       6        1  6.147099
10     0.013798  0.002649       7        1  5.918556
9      0.030235  0.004770       8        1  5.646717
3      0.020602  0.003571       9        1  5.632183
18     0.037747  0.005597      10        1  5.524553
11     0.033120  0.005097      11        1  5.162052
19     0.031066  0.004856      12        1  5.026505
4      0.036154  0.005429      13        1  4.941406
16     0.042050  0.006027      14        1  4.760108
17     0.049305  0.006590      15        1  2.818011
7      0.050468  0.006349      16        1  2.079047
6      0.050678  0.006599      17        1  1.616611
15     0.050368  0.006326      18        1  1.364301
13     0.047629  0.005790      19        1  0.864351
5      0.044801  0.005303      20        1  0.478876, topic_info=     Category         Freq                 Term        Total  loglift  logprob
term                                                                          
30    Default  5564.000000             chemical  5564.000000  30.0000  30.0000
140   Default  4220.000000              effexor  4220.000000  29.0000  29.0000
352   Default  6429.000000            impulsive  6429.000000  28.0000  28.0000
265   Default  5618.000000            complaint  5618.000000  27.0000  27.0000
150   Default  4484.000000          development  4484.000000  26.0000  26.0000
164   Default  4283.000000                 cunt  4283.000000  25.0000  25.0000
68    Default  3947.000000               status  3947.000000  24.0000  24.0000
159   Default  4170.000000            psychotic  4170.000000  23.0000  23.0000
194   Default  3890.000000               review  3890.000000  22.0000  22.0000
6     Default  3733.000000              persona  3733.000000  21.0000  21.0000
282   Default  3864.000000                 haze  3864.000000  20.0000  20.0000
239   Default  4057.000000                upper  4057.000000  19.0000  19.0000
50    Default  3732.000000               clingy  3732.000000  18.0000  18.0000
72    Default  3968.000000                asian  3968.000000  17.0000  17.0000
219   Default  4261.000000        disappointing  4261.000000  16.0000  16.0000
116   Default  4438.000000              unloved  4438.000000  15.0000  15.0000
372   Default  3397.000000        dysfunctional  3397.000000  14.0000  14.0000
119   Default  3302.000000             capacity  3302.000000  13.0000  13.0000
355   Default  4307.000000             overseas  4307.000000  12.0000  12.0000
221   Default  3941.000000              keeping  3941.000000  11.0000  11.0000
96    Default  3891.000000                  pre  3891.000000  10.0000  10.0000
18    Default  3245.000000               debate  3245.000000   9.0000   9.0000
179   Default  3137.000000               hunger  3137.000000   8.0000   8.0000
184   Default  4048.000000         surprisingly  4048.000000   7.0000   7.0000
298   Default  2219.000000           transition  2219.000000   6.0000   6.0000
328   Default  3419.000000                 spin  3419.000000   5.0000   5.0000
388   Default  3245.000000           complicate  3245.000000   4.0000   4.0000
49    Default  3211.000000                 wild  3211.000000   3.0000   3.0000
310   Default  3407.000000               submit  3407.000000   2.0000   2.0000
283   Default  3171.000000               injure  3171.000000   1.0000   1.0000
...       ...          ...                  ...          ...      ...      ...
8029  Topic20     0.152778              vicodon    10.716410   1.0909  -9.9584
7036  Topic20     0.152778             dedicate    10.718203   1.0908  -9.9584
7038  Topic20     0.152778